In [1]:
from __future__ import division

import config

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pickle
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.contrib.keras as kr

from helper import content_to_vector, class_to_vector
from collections import Counter

In [6]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
train = train.sample(len(train)).reset_index(drop=True)

In [11]:
from more_itertools import chunked

In [16]:
for i in chunked(range(len(train)), 64):
    train.loc[i]

,class,content
0,时尚,花边报：新蜜桃成熟时 潘霜霜挑战性感尤物导语：今年5月上映的香港情色片《3D肉蒲团之极乐宝鉴...
1,教育,过来人：改专业在美国捧上“铁饭碗”在国内，就听说过“哪几种人不宜到美国去”的传言，其中有一条...
2,娱乐,从《少林寺》到《新少林寺》 功夫片的变革之路“天下功夫出少林”。功夫片作为中国最具盛名、发展...
3,游戏,PC独占《绝境重启》新视频与截图公布8月3日，波兰开发商Flying Wild Hog公布了...
4,科技,美研究人员设计出人造发电细胞本报华盛顿10月19日电(记者毛黎)美国耶鲁大学研究人员表示，他...
5,游戏,史上最严肃吐槽：最像网游的单机游戏【新浪游戏专稿，转载请注明出处】前言：时代在发展，科技在进...
6,娱乐,《南京》坦克道具农民手工造 每台造价18万(图)本报讯(记者 薛姣) 行驶平稳、炮塔可360...
7,房产,楼市或将二次筑底 房地产企业勿盲目提价房地产市场成交量出现近3个月来首次环比下滑，呈现乍暖又...
8,家居,飞扬的青春 我要属于我的衣柜(组图)时尚衣柜，以丰富的色彩组合，演绎着时尚女性飞扬的青春、浪...
9,教育,国家“自费留学生奖学金”开始申请近日，国家留学基金管理委员会向社会公布2010年“国家优秀自...


In [ ]:
# sample_data = train.sample(1000)
# x_data = list(sample_data['content'].apply(content_to_vector))
# y_data = list(sample_data['class'].apply(class_to_vector))

In [ ]:
graph = tf.Graph()
tfcfg = tf.ConfigProto(intra_op_parallelism_threads=config.cpu_to_use)
session = tf.Session(graph=graph, config=tfcfg)
with session.graph.as_default():
    # Load model
    saver = tf.train.import_meta_graph(config.model_filename + '.meta')
    saver.restore(session, config.model_filename)
    
    # Load node information
    nodes = pickle.load(open(config.nodes_filename, "rU"))
    x = session.graph.get_tensor_by_name(nodes['x'])
    y = session.graph.get_tensor_by_name(nodes['y'])
    keep_prob = session.graph.get_tensor_by_name(nodes['keep_prob'])
    loss = session.graph.get_tensor_by_name(nodes['loss'])
    accuracy = session.graph.get_tensor_by_name(nodes['accuracy'])
    optimizer = session.graph.get_operation_by_name(nodes['optimizer'])    
    
    # Training ...
    for i, step in enumerate(range(1000), 1):
        # Load sample data
        sample_data = train.sample(10000)
        x_data = list(sample_data['content'].apply(content_to_vector))
        y_data = list(sample_data['class'].apply(class_to_vector))
        
        # Feed model
        begin_time = datetime.now()
        eval_list = [optimizer, loss, accuracy]
        _, l, a = session.run(eval_list, feed_dict={x: x_data, y: y_data, keep_prob: .75})
        
        print '%d: loss: %.3f, accuracy: %.3f, time:%.2fs' % (
            i, l, a, (datetime.now() - begin_time).total_seconds())
        
        # saver.save(session, config.model_filename)

INFO:tensorflow:Restoring parameters from model/model


In [ ]:
begin_time = datetime.now()

In [ ]:
1: loss: 2.303, accuracy: 0.002, time:38.15s
2: loss: 2.301, accuracy: 0.003, time:37.91s